In [1]:
import os
import csv
import numpy as np
from scipy import ndimage as ndi
from skimage import io, img_as_float
from skimage.filters import gabor_kernel
from skimage.transform import resize
from scipy.fftpack import dct as scipy_dct
import cv2
from skimage.feature import hog

In [ ]:
base_dir = os.path.expanduser("Datasets")
sub_dirs = ["NORMAL", "PNEUMONIA"]

TARGET_SIZE = (128, 128)

csv_file_path = os.path.expanduser("Datasets_features.csv")

kernels = [
    np.real(gabor_kernel(frequency, theta=theta / 4.0 * np.pi, sigma_x=sigma, sigma_y=sigma))
    for theta in range(4)
    for sigma in (1, 3)
    for frequency in (0.05, 0.25)
]

In [ ]:
def load_and_preprocess_image(image_path):
    image = img_as_float(io.imread(image_path, as_gray=True)) 
    resized_image = resize(image, TARGET_SIZE, anti_aliasing=True)
    return resized_image

def compute_gabor(image, kernels):
    feats = np.array([(ndi.convolve(image, kernel, mode='wrap').mean(), ndi.convolve(image, kernel, mode='wrap').var())
                      for kernel in kernels])
    return feats.flatten()

def compute_dct(image, num_coeffs=10):
    dct_flat = scipy_dct(scipy_dct(image, axis=0, norm='ortho'), axis=1, norm='ortho').flatten()
    return dct_flat[:num_coeffs]

def compute_ft(image, num_coeffs=10):
    fft_magnitude = np.abs(np.fft.fft2(image)).flatten()
    return np.sort(fft_magnitude)[-num_coeffs:]

def compute_phog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), pyramid_levels=3):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    pyramid_features = []
    for level in range(pyramid_levels):
        scaled_image = cv2.resize(image, (0, 0), fx=1 / (2 ** level), fy=1 / (2 ** level))
        hog_features = hog(scaled_image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=False, feature_vector=True)
        pyramid_features.extend(hog_features)
    return np.array(pyramid_features)

In [ ]:
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Numéro d'image", "Caractéristique[Gabor][DCT][Fourier][PHOG]", "Étiquette"])

    for sub_dir in sub_dirs:
        folder_path = os.path.join(base_dir, sub_dir)
        label = 0 if sub_dir == "NORMAL" else 1

        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(folder_path, filename)
                    
                    image = load_and_preprocess_image(image_path)
                    
                    gabor_feats = compute_gabor(image, kernels)
                    dct_feats = compute_dct(image, num_coeffs=20)
                    ft_feats = compute_ft(image, num_coeffs=20)
                    phog_feats = compute_phog(image)
                    
                    numero_image = filename.split('_')[-1].split('.')[0]
                    
                    features = [gabor_feats.tolist(), dct_feats.tolist(), ft_feats.tolist(), phog_feats.tolist()]

                    writer.writerow([numero_image, features, label])
                    print(f"Processed {filename}")

print(f"Image features saved to '{csv_file_path}'.")

Processed NORMAL_0.jpg
Processed NORMAL_1.jpg
Processed NORMAL_10.jpg
Processed NORMAL_11.jpg
Processed NORMAL_12.jpg
Processed NORMAL_13.jpg
Processed NORMAL_14.jpg
Processed NORMAL_15.jpg
Processed NORMAL_16.jpg
Processed NORMAL_17.jpg
Processed NORMAL_18.jpg
Processed NORMAL_19.jpg
Processed NORMAL_2.jpg
Processed NORMAL_20.jpg
Processed NORMAL_21.jpg
Processed NORMAL_22.jpg
Processed NORMAL_3.jpg
Processed NORMAL_4.jpg
Processed NORMAL_5.jpg
Processed NORMAL_6.jpg
Processed NORMAL_7.jpg
Processed NORMAL_8.jpg
Processed NORMAL_9.jpg
Processed PNEUMONIA_23.jpg
Processed PNEUMONIA_24.jpg
Processed PNEUMONIA_25.jpg
Processed PNEUMONIA_26.jpg
Processed PNEUMONIA_27.jpg
Processed PNEUMONIA_28.jpg
Processed PNEUMONIA_29.jpg
Processed PNEUMONIA_30.jpg
Processed PNEUMONIA_31.jpg
Processed PNEUMONIA_32.jpg
Processed PNEUMONIA_33.jpg
Processed PNEUMONIA_34.jpg
Processed PNEUMONIA_35.jpg
Processed PNEUMONIA_36.jpg
Processed PNEUMONIA_37.jpg
Processed PNEUMONIA_38.jpg
Processed PNEUMONIA_39.jpg